In [56]:
from konlpy.tag import Hannanum
# import jpype

In [57]:
hnn = Hannanum()

RuntimeError: Unable to start JVM at native/common/jp_env.cpp:60

In [3]:
print(hnn.analyze(u'롯데마트의 흑마늘 양념 치킨이 논란이 되고 있다.'))
print(hnn.morphs(u'롯데마트의 흑마늘 양념 치킨이 논란이 되고 있다.'))
print(hnn.nouns(u'다람쥐 헌 쳇바퀴에 타고파'))
print(hnn.pos(u'웃으면 더 행복합니다!'))

[[[('롯데마트', 'ncn'), ('의', 'jcm')], [('롯데마트의', 'ncn')], [('롯데마트', 'nqq'), ('의', 'jcm')], [('롯데마트의', 'nqq')]], [[('흑마늘', 'ncn')], [('흑마늘', 'nqq')]], [[('양념', 'ncn')]], [[('치킨', 'ncn'), ('이', 'jcc')], [('치킨', 'ncn'), ('이', 'jcs')], [('치킨', 'ncn'), ('이', 'ncn')]], [[('논란', 'ncpa'), ('이', 'jcc')], [('논란', 'ncpa'), ('이', 'jcs')], [('논란', 'ncpa'), ('이', 'ncn')]], [[('되', 'nbu'), ('고', 'jcj')], [('되', 'nbu'), ('이', 'jp'), ('고', 'ecc')], [('되', 'nbu'), ('이', 'jp'), ('고', 'ecs')], [('되', 'nbu'), ('이', 'jp'), ('고', 'ecx')], [('되', 'paa'), ('고', 'ecc')], [('되', 'paa'), ('고', 'ecs')], [('되', 'paa'), ('고', 'ecx')], [('되', 'pvg'), ('고', 'ecc')], [('되', 'pvg'), ('고', 'ecs')], [('되', 'pvg'), ('고', 'ecx')], [('되', 'px'), ('고', 'ecc')], [('되', 'px'), ('고', 'ecs')], [('되', 'px'), ('고', 'ecx')]], [[('있', 'paa'), ('다', 'ef')], [('있', 'px'), ('다', 'ef')]], [[('.', 'sf')], [('.', 'sy')]]]
['롯데마트', '의', '흑마늘', '양념', '치킨', '이', '논란', '이', '되', '고', '있', '다', '.']
['다람쥐', '쳇바퀴', '타고파']
[('웃', 'P'), ('으면', 'E'), 

In [1]:
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy.pool import NullPool
import matplotlib as plt
import matplotlib.pylab as plt
import seaborn as sns
from tqdm import tqdm
from PIL import Image
import requests
from io import BytesIO
# q = '''
# SELECT id,
#        uid,
#        name,
#        keyword,
#        category_id
# FROM service1_quicket.product_info
# WHERE left(category_id, 3) IN (310, 320, 400) and status = 0
# '''

# df = pd.read_sql(q, con=bun_dw)

In [2]:
df = pd.read_csv('product_info.csv')

/Users/emilyPark/.pyenv/versions/bun-analytics/lib/python3.7/site-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0,1) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [59]:
df.shape

(7836703, 5)

In [5]:
df.head()

,id,uid,name,keyword,category_id
0,100002112,5016511,[66] 끌리오 자켓,"66,끌리오,자켓",310100999.0
1,100013888,7680369,🔥당일배송가능🔥(260~280)구찌 모던 로퍼,"구지,구지신발,구찌신발,구찌슬리퍼,구지로퍼",400051300.0
2,100014144,4429338,⭐️구* 클러치⭐️,"구찌,구찌클러치,구찌클러치백,구찌가방,구찌핸드백",400020400.0
3,100014912,3550416,the izzat 봄 트위드자켓,"트위드자켓,트위드코트,봄코트,봄자켓,봄아우터",310100030.0
4,100021056,5811221,▷에르메스 오즈 켈리 뮬 로퍼 6color,"에르메스오즈,에르메스오즈뮬,에르메스켈리,에르메스로퍼,에르메스뮬",400040300.0


In [3]:
import re

def preprocess(name):
    name = str(name)
    regrex_pattern = re.compile(pattern = "["
    u"\U0001F600-\U0001F64F"  
    u"\U0001F300-\U0001F5FF"  
    u"\U0001F680-\U0001F6FF"  
    u"\U0001F1E0-\U0001F1FF"  
                       "]+", flags = re.UNICODE)
    name = regrex_pattern.sub(r'',name)
    name = re.sub(r'[^\w\s]', ' ', name)
    name = re.sub(r'\s+', ' ', name)
    return name.lower().strip()

In [5]:
df['preprocessed'] = df.name.map(preprocess)

In [6]:
df.head()

,id,uid,name,keyword,category_id,preprocessed
0,100002112,5016511,[66] 끌리오 자켓,"66,끌리오,자켓",310100999.0,66 끌리오 자켓
1,100013888,7680369,🔥당일배송가능🔥(260~280)구찌 모던 로퍼,"구지,구지신발,구찌신발,구찌슬리퍼,구지로퍼",400051300.0,당일배송가능 260 280 구찌 모던 로퍼
2,100014144,4429338,⭐️구* 클러치⭐️,"구찌,구찌클러치,구찌클러치백,구찌가방,구찌핸드백",400020400.0,구 클러치
3,100014912,3550416,the izzat 봄 트위드자켓,"트위드자켓,트위드코트,봄코트,봄자켓,봄아우터",310100030.0,the izzat 봄 트위드자켓
4,100021056,5811221,▷에르메스 오즈 켈리 뮬 로퍼 6color,"에르메스오즈,에르메스오즈뮬,에르메스켈리,에르메스로퍼,에르메스뮬",400040300.0,에르메스 오즈 켈리 뮬 로퍼 6color


In [14]:
hnn.analyze('무신사에서 구매한 트러커 자켓 팝니다 사이즈s')

[[[('무', 'ncn'), ('신사', 'ncr'), ('에서', 'jca')],
  [('무신사', 'ncn'), ('에서', 'jca')],
  [('무신사에', 'ncn'), ('서', 'jca')],
  [('무신사에서', 'ncn')],
  [('무', 'nqq'), ('신사', 'ncr'), ('에서', 'jca')],
  [('무신사', 'nqq'), ('에서', 'jca')],
  [('무신사에', 'nqq'), ('서', 'jca')],
  [('무신사에서', 'nqq')]],
 [[('구매', 'ncpa'), ('한', 'ncn')],
  [('구매', 'ncpa'), ('하', 'xsva'), ('ㄴ', 'etm')]],
 [[('트러커', 'ncn')], [('트러커', 'nqq')]],
 [[('자켓', 'ncn')]],
 [[('팝니다', 'ncn')],
  [('팔', 'pvg'), ('ㅂ니다', 'ef')],
  [('파', 'pvg'), ('ㅂ니다', 'ef')]],
 [[('사이즈s', 'ncn')], [('사이즈s', 'nqq')]]]

In [15]:
hnn.morphs('무신사에서 구매한 트러커 자켓 팝니다 사이즈s')

['무신사', '에서', '구매', '하', 'ㄴ', '트러커', '자켓', '팔', 'ㅂ니다', '사이즈s']

In [8]:
from konlpy.tag import Kkma
kkma = Kkma()

In [13]:
kkma.morphs('무신사에서 구매한 트러커 자켓 팝니다 사이즈s')

['무',
 '신사',
 '에서',
 '구매',
 '하',
 'ㄴ',
 '트',
 '러',
 '커',
 '어',
 '자켓',
 '팔',
 'ㅂ니다',
 '사',
 '이즈',
 's']

In [ ]:
['무', '신사', '에서', '구매', '하', 'ㄴ', '트', '러', '커', '어', '자켓', '팔', 'ㅂ니다', '사', '이즈', 's']

In [14]:
print(hnn.morphs('무신사에서 구매한 트러커 자켓 팝니다 사이즈s'))
print(hnn.morphs('프라다 숄더백'))
print(hnn.morphs('검정 코튼 버튼 오버핏 후드'))

['무신사', '에서', '구매', '하', 'ㄴ', '트러커', '자켓', '팔', 'ㅂ니다', '사이즈s']
['프라', '이', '다', '숄더백']
['검정', '코튼', '버튼', '오버핏', '후드']


In [15]:
print(kkma.morphs('무신사에서 구매한 트러커 자켓 팝니다 사이즈s'))
print(kkma.morphs('프라다 숄더백'))
print(kkma.morphs('검정 코튼 버튼 오버핏 후드'))

['무', '신사', '에서', '구매', '하', 'ㄴ', '트', '러', '커', '어', '자켓', '팔', 'ㅂ니다', '사', '이즈', 's']
['프라다', '숄더백']
['검정', '코튼', '버튼', '오버', '핏', '후드']


!pip install python-mecab
https://github.com/jeongukjae/python-mecab

In [1]:
from mecab import Tagger

In [ ]:
tagger = Tagger()
tagger.parse('무신사에서 구매한 트러커 자켓 팝니다 사이즈s')

In [2]:
# !pip install sentencepiece

In [7]:
import sentencepiece as spm

In [16]:
with open('product_info_name.txt', 'w', encoding='utf-8') as f:
    f.write('\n'.join(df['preprocessed']))

In [16]:
with open('product_info_name_wo_preprocess.txt', 'w', encoding='utf-8') as f:
    f.write('\n'.join(df['name'].apply(lambda x: str(x).strip().lower())))

In [43]:
df['name_keyword_preprocessed'] = (df['name'] + ' ' + df['keyword']).map(preprocess)

In [45]:
with open('product_info_name_and_keyword.txt', 'w', encoding='utf-8') as f:
    f.write('\n'.join(df['name_keyword_preprocessed'].apply(lambda x: str(x).strip().lower())))

In [12]:
max(df['preprocessed'].apply(lambda x: len(x)))

94

In [40]:
spm.SentencePieceTrainer.Train('--input=product_info_name.txt --model_prefix=p_name --vocab_size=10000 --model_type=bpe --max_sentence_length=100')

input : 학습시킬 파일
model_prefix : 만들어질 모델 이름
vocab_size : 단어 집합의 크기
model_type : 사용할 모델 (unigram(default), bpe, char, word)
max_sentence_length: 문장의 최대 길이
pad_id, pad_piece: pad token id, 값
unk_id, unk_piece: unknown token id, 값
bos_id, bos_piece: begin of sentence token id, 값
eos_id, eos_piece: end of sequence token id, 값
user_defined_symbols: 사용자 정의 토큰

In [29]:
sp1 = spm.SentencePieceProcessor()
vocab_file = "p_name.model"
sp1.load(vocab_file)

True

In [17]:
spm.SentencePieceTrainer.Train('--input=product_info_name_wo_preprocess.txt --model_prefix=p_name2 --vocab_size=10000 --model_type=bpe --max_sentence_length=100')

In [21]:
sp2 = spm.SentencePieceProcessor()
vocab_file = "p_name2.model"
sp2.load(vocab_file)

True

In [46]:
spm.SentencePieceTrainer.Train('--input=product_info_name_and_keyword.txt --model_prefix=p_name3 --vocab_size=10000 --model_type=bpe --max_sentence_length=100')

In [47]:
sp3 = spm.SentencePieceProcessor()
vocab_file = "p_name3.model"
sp3.load(vocab_file)

True

In [18]:
# ver1
lines = [
  "무신사에서 구매한 트러커 자켓 팝니다 사이즈s",
  "프라다 숄더백"
]
for line in lines:
    print(line)
    print(sp.encode_as_pieces(line))
    print(sp.encode_as_ids(line))
    print()

무신사에서 구매한 트러커 자켓 팝니다 사이즈s
['▁무신사', '에서', '▁구매', '한', '▁트러커', '▁자켓', '▁팝니다', '▁사이즈', 's']
[2894, 2345, 1128, 3784, 3239, 44, 183, 280, 3650]

프라다 숄더백
['▁프라다', '▁숄더백']
[336, 234]



In [35]:
df[13423:13440]

,id,uid,name,keyword,category_id,preprocessed
13423,9632178,366440,디젤자타니연청 구하시는분,NaN,320110999.0,디젤자타니연청 구하시는분
13424,9641010,574961,원피스,애플,310120999.0,원피스
13425,101770689,4605178,루이비통 백팩,"루이비통,루이비통가방,루이비통백팩,명품백팩",400010500.0,루이비통 백팩
13426,101776065,7739066,#프라다#여권지갑#,"프라다,여권지갑",400060999.0,프라다 여권지갑
13427,100133708,7298480,원더플레이스 카라달린 긴팔,"원더플레이스,셔츠,긴팔티,현장학습,난방",310010999.0,원더플레이스 카라달린 긴팔
13428,100138572,3890550,(정품/명품) 100만원 넘는 스텔라메카트니 화이트진,"허리26,스텔라메카트니,명품,진품,새상품",310140010.0,정품 명품 100만원 넘는 스텔라메카트니 화이트진
13429,99569036,1394027,치마,치마,310130999.0,치마
13430,101778113,4978210,새상품🖤린넨오프숄더 원피스 청순! 너무예뻐요 ㅜㅠ재고2개뿐,"린넨원피스,데일리원피스,롱원피스,데이트원피스,청순원피스",310120030.0,새상품린넨오프숄더 원피스 청순 너무예뻐요 ㅜㅠ재고2개뿐
13431,101779905,4666270,에스쁘렌도 핑크리본 245-4,"구제신발가게,중고신발,구제신발,여성화,샌달",400050100.0,에스쁘렌도 핑크리본 245 4
13432,101780673,5493386,【쉬프트】 [0080] 크롬하츠 남여공용 925은 목걸이,"크롬하츠,목걸이,크롬하츠목걸이,패션목걸이",400081300.0,쉬프트 0080 크롬하츠 남여공용 925은 목걸이


In [39]:
# ver1
lines = [
  "sale 펀칭 주름스커트 15000 7000",
  "스파이더 티셔츠 반목티",
  "free xoxo 반팔",
  "chanel 샤넬 화장품 파우치",
  "남자 기본 베이직 레이어드 오버핏 반팔 티셔츠 2color",
  "all 크롬하츠 아베크롬비",
  "새상품린넨오프숄더 원피스 청순 너무예뻐요 ㅜㅠ재고2개뿐"
]
for line in lines:
    print(line)
    print(sp1.encode_as_pieces(line))
    print(sp1.encode_as_ids(line))
    print()

sale 펀칭 주름스커트 15000 7000
['▁sale', '▁펀칭', '▁주름', '스커트', '▁15000', '▁7', '000']
[1632, 1332, 1085, 139, 3134, 185, 348]

스파이더 티셔츠 반목티
['▁스파이더', '▁티셔츠', '▁반목', '티']
[1189, 117, 3067, 3902]

free xoxo 반팔
['▁free', '▁x', 'o', 'x', 'o', '▁반팔']
[44, 123, 3930, 3998, 3930, 55]

chanel 샤넬 화장품 파우치
['▁chanel', '▁샤넬', '▁화', '장', '품', '▁파우치']
[2520, 201, 1790, 4008, 3906, 746]

남자 기본 베이직 레이어드 오버핏 반팔 티셔츠 2color
['▁남자', '▁기본', '▁베이직', '▁레이어드', '▁오버핏', '▁반팔', '▁티셔츠', '▁2', 'color']
[197, 625, 860, 878, 394, 55, 117, 4, 309]

all 크롬하츠 아베크롬비
['▁all', '▁크롬하츠', '▁아베크롬비']
[325, 965, 834]

새상품린넨오프숄더 원피스 청순 너무예뻐요 ㅜㅠ재고2개뿐
['▁새상품', '린넨', '오프', '숄더', '▁원피스', '▁청순', '▁너무', '예', '뻐요', '▁', 'ᅮ', 'ᅲ', '재고', '2', '개', '뿐']
[28, 295, 1180, 116, 21, 3286, 3425, 4213, 2766, 3887, 4670, 4473, 3717, 3896, 4126, 4635]



In [49]:
# ver3
lines = [
  "sale 펀칭 주름스커트 15000 7000",
  "스파이더 티셔츠 반목티",
  "free xoxo 반팔",
  "chanel 샤넬 화장품 파우치",
  "남자 기본 베이직 레이어드 오버핏 반팔 티셔츠 2color",
  "all 크롬하츠 아베크롬비",
  "새상품린넨오프숄더 원피스 청순 너무예뻐요 ㅜㅠ재고2개뿐"
]
for line in lines:
    print(line)
    print(sp3.encode_as_pieces(line))
    print(sp3.encode_as_ids(line))
    print()

sale 펀칭 주름스커트 15000 7000
['▁sale', '▁펀칭', '▁주름스커트', '▁15000', '▁7000']
[2889, 1517, 3742, 4267, 7630]

스파이더 티셔츠 반목티
['▁스파이더', '▁티셔츠', '▁반목', '티']
[953, 160, 2939, 8942]

free xoxo 반팔
['▁free', '▁x', 'o', 'x', 'o', '▁반팔']
[86, 150, 8997, 9052, 8997, 61]

chanel 샤넬 화장품 파우치
['▁chanel', '▁샤넬', '▁화장품', '▁파우치']
[2004, 131, 4083, 812]

남자 기본 베이직 레이어드 오버핏 반팔 티셔츠 2color
['▁남자', '▁기본', '▁베이직', '▁레이어드', '▁오버핏', '▁반팔', '▁티셔츠', '▁2', 'color']
[127, 522, 1178, 1000, 427, 61, 160, 12, 1056]

all 크롬하츠 아베크롬비
['▁all', '▁크롬하츠', '▁아베크롬비']
[636, 1017, 977]

새상품린넨오프숄더 원피스 청순 너무예뻐요 ㅜㅠ재고2개뿐
['▁새상품', '린넨', '오프숄더', '▁원피스', '▁청순', '▁너무', '예', '뻐요', '▁', 'ᅮ', 'ᅲ', '재고', '2', '개', '뿐']
[85, 240, 4264, 26, 3087, 5094, 9219, 2353, 8926, 9816, 9634, 7719, 8943, 9215, 9593]



In [38]:
# ver2
lines = [
  "sale 펀칭 주름스커트 15000 7000",
  "# 스파이더# 티셔츠# 반목티#",
  "[free] xoxo 반팔",
  "chanel 샤넬 화장품 파우치",
  "남자 기본 베이직 레이어드 오버핏 반팔 티셔츠 2color",
  "[all] 크롬하츠 아베크롬비",
  "새상품🖤린넨오프숄더 원피스 청순! 너무예뻐요 ㅜㅠ재고2개뿐"
]
for line in lines:
    print(line)
    print(sp2.encode_as_pieces(line))
    print(sp2.encode_as_ids(line))
    print()

sale 펀칭 주름스커트 15000 7000
['▁sale', '▁펀칭', '▁주름', '스커트', '▁15', '000', '▁7', '000']
[2335, 1444, 1172, 120, 1332, 343, 328, 343]

# 스파이더# 티셔츠# 반목티#
['▁#', '▁스파이더', '#', '▁티셔츠', '#', '▁반목', '티', '#']
[362, 1326, 3851, 109, 3851, 3364, 3623, 3851]

[free] xoxo 반팔
['▁[', 'free', ']', '▁x', 'o', 'x', 'o', '▁반팔']
[3, 37, 3610, 426, 3651, 3725, 3651, 45]

chanel 샤넬 화장품 파우치
['▁chanel', '▁샤넬', '▁화', '장', '품', '▁파우치']
[3120, 214, 2072, 3735, 3627, 798]

남자 기본 베이직 레이어드 오버핏 반팔 티셔츠 2color
['▁남자', '▁기본', '▁베이직', '▁레이어드', '▁오버핏', '▁반팔', '▁티셔츠', '▁2', 'color']
[229, 657, 917, 941, 393, 45, 109, 15, 286]

[all] 크롬하츠 아베크롬비
['▁[', 'all', ']', '▁크롬하츠', '▁아베크롬비']
[3, 281, 3610, 1097, 904]

새상품🖤린넨오프숄더 원피스 청순! 너무예뻐요 ㅜㅠ재고2개뿐
['▁새상품', '🖤', '린넨', '오프숄더', '▁원피스', '▁청', '순', '!', '▁너', '무', '예', '뻐요', '▁', 'ᅮ', 'ᅲ', '재고', '2', '개', '뿐']
[74, 4390, 276, 3033, 20, 79, 4099, 3705, 2855, 3719, 3953, 2906, 3604, 4448, 4226, 2825, 3615, 3860, 4410]



In [20]:
sp.GetPieceSize()

5000

In [21]:
sp.IdToPiece(430)

'브라운'

In [23]:
sp.PieceToId('브라운')

430

In [24]:
sp.DecodeIds([41, 141, 1364, 1120, 4, 666, 285, 92, 1078, 33, 91])

'나이키티지파드 스웨터 2뷔스티에아요 바 경량 정품 레'

In [28]:
sp.DecodePieces(['▁무신사', '에서', '▁구매', '한', '▁트러커', '▁자켓', '▁팝니다', '▁사이즈', 's'])

'무신사에서 구매한 트러커 자켓 팝니다 사이즈s'

In [53]:
with open('./p_name.vocab', encoding='utf-8') as f:
    Vo1 = [doc.strip().split("\t") for doc in f]

# w[0]: token name    
# w[1]: token score
word2idx1 = {w[0]: i for i, w in enumerate(Vo1)}

In [54]:
with open('./p_name3.vocab', encoding='utf-8') as f:
    Vo3 = [doc.strip().split("\t") for doc in f]

# w[0]: token name    
# w[1]: token score
word2idx3 = {w[0]: i for i, w in enumerate(Vo3)}

In [55]:
word2idx1

{'<unk>': 0,
 '<s>': 1,
 '</s>': 2,
 '▁스': 3,
 '▁2': 4,
 '피스': 5,
 '원피스': 6,
 '▁1': 7,
 '▁블': 8,
 '▁가': 9,
 '자켓': 10,
 '니트': 11,
 '셔츠': 12,
 '▁새': 13,
 '▁아': 14,
 '▁여': 15,
 '▁반': 16,
 '▁남': 17,
 '▁나': 18,
 '▁m': 19,
 '바지': 20,
 '▁원피스': 21,
 '상품': 22,
 '▁s': 23,
 '▁정': 24,
 '레이': 25,
 '▁니트': 26,
 '▁f': 27,
 '▁새상품': 28,
 '이즈': 29,
 '▁후': 30,
 '▁여성': 31,
 're': 32,
 '▁정품': 33,
 '코트': 34,
 '00': 35,
 '▁구': 36,
 '사이즈': 37,
 '▁5': 38,
 '▁나이': 39,
 '패딩': 40,
 '▁나이키': 41,
 '▁오': 42,
 'ree': 43,
 '▁free': 44,
 '▁미': 45,
 '▁9': 46,
 '우스': 47,
 '▁3': 48,
 '▁크': 49,
 '▁블랙': 50,
 '니다': 51,
 '▁자켓': 52,
 '▁트': 53,
 '라우스': 54,
 '▁반팔': 55,
 '▁롱': 56,
 '▁후드': 57,
 '▁l': 58,
 '맨투': 59,
 '맨투맨': 60,
 '커트': 61,
 '▁카': 62,
 '티셔츠': 63,
 '▁에': 64,
 '▁프': 65,
 '▁루': 66,
 '▁라': 67,
 '디건': 68,
 '이비': 69,
 '지갑': 70,
 '걸이': 71,
 '▁체': 72,
 '▁맨투맨': 73,
 '다스': 74,
 '디다스': 75,
 '▁청': 76,
 '▁a': 77,
 '▁아디다스': 78,
 '▁플': 79,
 '팬츠': 80,
 '▁로': 81,
 '▁무': 82,
 '▁55': 83,
 '▁베': 84,
 '▁디': 85,
 '▁블라우스': 86,
 '집업': 87,
 '▁

In [61]:
# size up the vocab size


spm.SentencePieceTrainer.Train('--input=product_info_name.txt --model_prefix=name_bpe_20000 --vocab_size=20000 --model_type=bpe --max_sentence_length=100')

In [62]:
sp_bpe_20000 = spm.SentencePieceProcessor()
vocab_file = "name_bpe_20000.model"
sp_bpe_20000.load(vocab_file)

True

In [41]:
t = spm.SentencePieceProcessor()
vocab_file = "name_bpe_20000.model"
t.load(vocab_file)

True

In [12]:
df['preprocessed'].to_csv('preprocessed_name.csv', index=False)

In [13]:
names = pd.read_csv('preprocessed_name.csv')

In [28]:
names['preprocessed']

0                         66 끌리오 자켓
1           당일배송가능 260 280 구찌 모던 로퍼
2                             구 클러치
3                 the izzat 봄 트위드자켓
4            에르메스 오즈 켈리 뮬 로퍼 6color
                     ...           
7836698    지오다노 남성 린넨 셔츠 xl 105 사이즈
7836699                 정품 나이키 져지 l
7836700                  핸드메이드 헤어밴드
7836701      오프화이트 16ss 플란넬셔츠 쿨매합니다
7836702                  잭앤존스 스컬 반팔
Name: preprocessed, Length: 7836703, dtype: object

In [30]:
[str(sp_unigram_20000.encode_as_pieces(str(name)))[1:-1] for name in names['preprocessed'][:10]]

["'▁66', '▁끌', '리오', '▁자켓'",
 "'▁당일배송', '가능', '▁260', '▁280', '▁구찌', '▁모던', '▁로퍼'",
 "'▁구', '▁클러치'",
 "'▁the', '▁izzat', '▁봄', '▁트위드자켓'",
 "'▁에르메스', '▁오즈', '▁켈리', '▁뮬', '▁로퍼', '▁6', 'color'",
 "'▁플랙진', '▁베를린', '▁청바지', '▁사이즈', '28', '▁28', '▁608'",
 "'▁s', 'tra', 'w', 'berry', '▁field', 's', '▁스커트'",
 "'▁구찌스타일', '킬힐', '225', '240'",
 "'▁asos', '플랫슈즈', '▁250'",
 "'▁원피스'"]

In [63]:
lines = [
  "sale 펀칭 주름스커트 15000 7000",
  "스파이더 티셔츠 반목티",
  "free xoxo 반팔",
  "chanel 샤넬 화장품 파우치",
  "남자 기본 베이직 레이어드 오버핏 반팔 티셔츠 2color",
  "all 크롬하츠 아베크롬비",
  "새상품린넨오프숄더 원피스 청순 너무예뻐요 ㅜㅠ재고2개뿐"
]
for line in lines:
    print(line)
    print(sp_bpe_20000.encode_as_pieces(line))
    print(sp_bpe_20000.encode_as_ids(line))
    print()

sale 펀칭 주름스커트 15000 7000
['▁sale', '▁펀칭', '▁주름스커트', '▁15000', '▁7000']
[1632, 1332, 5983, 3134, 5511]

스파이더 티셔츠 반목티
['▁스파이더', '▁티셔츠', '▁반목티']
[1189, 117, 16194]

free xoxo 반팔
['▁free', '▁x', 'ox', 'o', '▁반팔']
[44, 123, 10526, 18930, 55]

chanel 샤넬 화장품 파우치
['▁chanel', '▁샤넬', '▁화장품', '▁파우치']
[2520, 201, 5532, 746]

남자 기본 베이직 레이어드 오버핏 반팔 티셔츠 2color
['▁남자', '▁기본', '▁베이직', '▁레이어드', '▁오버핏', '▁반팔', '▁티셔츠', '▁2', 'color']
[197, 625, 860, 878, 394, 55, 117, 4, 309]

all 크롬하츠 아베크롬비
['▁all', '▁크롬하츠', '▁아베크롬비']
[325, 965, 834]

새상품린넨오프숄더 원피스 청순 너무예뻐요 ㅜㅠ재고2개뿐
['▁새상품', '린넨', '오프숄더', '▁원피스', '▁청순', '▁너무', '예뻐요', '▁ᅮ', 'ᅲ', '재고', '2', '개', '뿐']
[28, 295, 4068, 21, 3286, 3425, 11171, 14752, 19473, 3717, 18896, 19126, 19635]



In [8]:
spm.SentencePieceTrainer.Train('--input=product_info_name.txt --model_prefix=name_unigram_20000 --vocab_size=20000 --model_type=unigram --max_sentence_length=100')

In [20]:
sp_unigram_20000 = spm.SentencePieceProcessor()
vocab_file = "name_unigram_20000.model"
sp_unigram_20000.load(vocab_file)

True

In [10]:
df['preprocessed'][:100]

,id,uid,name,keyword,category_id,preprocessed
0,100002112,5016511,[66] 끌리오 자켓,"66,끌리오,자켓",310100999.0,66 끌리오 자켓
1,100013888,7680369,🔥당일배송가능🔥(260~280)구찌 모던 로퍼,"구지,구지신발,구찌신발,구찌슬리퍼,구지로퍼",400051300.0,당일배송가능 260 280 구찌 모던 로퍼
2,100014144,4429338,⭐️구* 클러치⭐️,"구찌,구찌클러치,구찌클러치백,구찌가방,구찌핸드백",400020400.0,구 클러치
3,100014912,3550416,the izzat 봄 트위드자켓,"트위드자켓,트위드코트,봄코트,봄자켓,봄아우터",310100030.0,the izzat 봄 트위드자켓
4,100021056,5811221,▷에르메스 오즈 켈리 뮬 로퍼 6color,"에르메스오즈,에르메스오즈뮬,에르메스켈리,에르메스로퍼,에르메스뮬",400040300.0,에르메스 오즈 켈리 뮬 로퍼 6color


In [66]:
lines = [
  "sale 펀칭 주름스커트 15000 7000",
  "스파이더 티셔츠 반목티",
  "free xoxo 반팔",
  "chanel 샤넬 화장품 파우치",
  "남자 기본 베이직 레이어드 오버핏 반팔 티셔츠 2color",
  "all 크롬하츠 아베크롬비",
  "새상품린넨오프숄더 원피스 청순 너무예뻐요 ㅜㅠ재고2개뿐"
]
for line in lines:
    print(line)
    print(sp_unigram_20000.encode_as_pieces(line))
    print(sp_unigram_20000.encode_as_ids(line))
    print()

sale 펀칭 주름스커트 15000 7000
['▁sale', '▁펀칭', '▁주름스커트', '▁15000', '▁7000']
[835, 757, 4263, 1888, 3802]

스파이더 티셔츠 반목티
['▁스파이더', '▁티셔츠', '▁반목티']
[566, 25, 13557]

free xoxo 반팔
['▁free', '▁xoxo', '▁반팔']
[9, 18792, 28]

chanel 샤넬 화장품 파우치
['▁chanel', '▁샤넬', '▁화장품', '▁파우치']
[1415, 51, 4441, 321]

남자 기본 베이직 레이어드 오버핏 반팔 티셔츠 2color
['▁남자', '▁기본', '▁베이직', '▁레이어드', '▁오버핏', '▁반팔', '▁티셔츠', '▁2', 'color']
[55, 279, 405, 399, 118, 28, 25, 27, 87]

all 크롬하츠 아베크롬비
['▁all', '▁크롬하츠', '▁아베크롬비']
[90, 435, 365]

새상품린넨오프숄더 원피스 청순 너무예뻐요 ㅜㅠ재고2개뿐
['▁새상품', '린넨', '오프숄더', '▁원피스', '▁청순', '▁너무', '예뻐요', '▁ᅮ', 'ᅲ', '재고', '2', '개', '뿐']
[4, 1864, 3020, 3, 2194, 2633, 8932, 12188, 4665, 6708, 41, 269, 9313]



In [75]:
brand = pd.read_csv('./brand extraction dictionary - final.csv')

In [76]:
brand = brand.iloc[:, :3]

In [77]:
brand.columns = ['name', 'eng', 'syn']

In [81]:
brands = brand['name'].tolist() + brand['eng'].tolist() + brand['syn'].tolist()

In [85]:
brands = list(set(brands))

In [86]:
len(brands)

713

In [95]:
brand_set = set()
for b in brands:
    l = str(b).split(',')
    for i in l:
        brand_set.add(i.strip())

In [97]:
len(brand_set)

747

In [100]:
brand_set

{'JOSEPH&STACEY',
 'MCM',
 'a cold wall',
 'a.p.c.',
 'abercrombie',
 'abercrombie & fitch',
 'acmedelavie',
 'acne',
 'acne studio',
 'ader error',
 'adidas',
 'adlv',
 'ajobyajo',
 'alden',
 'alexander mcqueen',
 'alexander wang',
 'all saints',
 'allegri',
 'alpha industries',
 'alyx',
 'ami',
 'amiri',
 'andar',
 'andersen andersen',
 'andersson bell',
 'another office',
 "arc'teryx",
 'archivepke',
 'asics',
 'audemars piguet',
 'avirex',
 'axel arigato',
 'balenciaga',
 'ballantyne',
 'balmain',
 'bape',
 'baracuta',
 'barbour',
 'beams',
 'beanpole',
 'belier',
 'berluti',
 'birthdaysuit',
 'black yak',
 'blankof',
 'bless',
 'bottega veneta',
 'boucheron',
 'breitling',
 'brooks brothers',
 'burberry',
 'buzz rickson',
 'bvlgari',
 'callaway',
 'calvin klein',
 'camper',
 'canada goose',
 'carhartt',
 'carlyn',
 'cartier',
 'casio',
 'castel bajac',
 'celine',
 'champion',
 'chanel',
 'charles & keith',
 'chaumet',
 'chloe',
 'chrome hearts',
 'citizen',
 'ck',
 'clarks',
 'coa

In [87]:
with open('./name_bpe_20000.vocab', encoding='utf-8') as f:
    name_bpe = [doc.strip().split("\t") for doc in f]

name_bpe_dict = {w[0]: i for i, w in enumerate(name_bpe)}

In [88]:
with open('./name_unigram_20000.vocab', encoding='utf-8') as f:
    name_unigram = [doc.strip().split("\t") for doc in f]

name_unigram_dict = {w[0]: i for i, w in enumerate(name_unigram)}

In [98]:
bpe_list = []
uni_list = []
for b in brand_set:
    if b in name_bpe_dict:
        bpe_list.append(b)
    if b in name_unigram_dict:
        uni_list.append(b)


In [99]:
print('bpe: {}'.format(len(bpe_list)))
print('unigram: {}'.format(len(uni_list)))

bpe: 65
unigram: 99
